In [ ]:
from google_play_scraper import reviews
import pandas as pd
import time

# App package name
app_id = "com.msf.sbicap.securities"

all_reviews = []
token = None
retry_count = 0
max_reviews = 15000  # Limit to 15k reviews

while len(all_reviews) < max_reviews:
    try:
        result, token = reviews(
            app_id,
            lang='en',               # Language
            country='in',            # Country
            count=200,               # 200 per request
            continuation_token=token
        )

        all_reviews.extend(result)
        print(f" Fetched {len(all_reviews)} reviews so far...")

        if not token or len(all_reviews) >= max_reviews:
            print(" Reached the review limit or no more reviews available.")
            break

        retry_count = 0
        time.sleep(1)

    except Exception as e:
        retry_count += 1
        print(f"⚠ Error: {e}. Retrying {retry_count}/3...")
        if retry_count >= 3:
            print(" Stopping after 3 retries.")
            break
        time.sleep(2)


all_reviews = all_reviews[:max_reviews]

# Converting to DataFrame
df = pd.DataFrame(all_reviews)

# Selecting only required columns
df = df[['userName', 'content', 'score', 'at']]

# Save to Excel
df.to_excel("sbi_reviews_15k.xlsx", index=False)

print(f" Exported {len(df)} reviews to sbi_reviews_15k.xlsx")
